In [1]:
# Install and import necessary libraries
!pip install spotipy
!pip install pylast
import requests
import pandas as pd
import numpy as np
import pylast
import concurrent.futures
from tqdm.notebook import tqdm
import time

In [2]:
# Set my credentials
CLIENT_ID = "9f8fa04c80a94cbc99b0190449fe7374"
CLIENT_SECRET = "0169432afd76420a8b992fd89788610b"


In [3]:
# Import spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,
                                                           client_secret=CLIENT_SECRET))

In [4]:
# Load DataFrame from pickle file
songs_apis_df = pd.read_pickle('sorted_df.pkl')

In [5]:
# Show the entire DataFrame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
songs_apis_df 

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,in_shazam_charts,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Blinding Lights,The Weeknd,1,2019,11,29,43899,69,3703895074,672,199,"3,421",20,NaN,171,C#,Major,50,38,80,0,0,9,7
1,Shape of You,Ed Sheeran,1,2017,1,6,32181,10,3562543890,33,0,"6,808",7,0,96,C#,Minor,83,93,65,58,0,9,8
2,Someone You Loved,Lewis Capaldi,1,2018,11,8,17836,53,2887241814,440,125,"1,800",0,NaN,110,C#,Major,50,45,41,75,0,11,3
3,Dance Monkey,Tones and I,1,2019,5,10,24529,0,2864791672,533,167,"3,595",6,NaN,98,F#,Minor,82,54,59,69,0,18,10
4,Sunflower - Spider-Man: Into the Spider-Verse,"Post Malone, Swae Lee",2,2018,10,9,24094,78,2808096550,372,117,843,4,69,90,D,Major,76,91,50,54,0,7,5
5,One Dance,"Drake, WizKid, Kyla",3,2016,4,4,43257,24,2713922350,433,107,"3,631",0,26,104,C#,Major,77,36,63,1,0,36,5
6,STAY (with Justin Bieber),"Justin Bieber, The Kid Laroi",2,2021,7,9,17050,36,2665343922,492,99,798,31,0,170,C#,Major,59,48,76,4,0,10,5
7,Believer,Imagine Dragons,1,2017,1,31,18986,23,2594040133,250,121,"2,969",10,31,125,A#,Minor,77,74,78,4,0,23,11
8,Closer,"The Chainsmokers, Halsey",2,2016,5,31,28032,0,2591224264,315,159,"2,179",0,44,95,G#,Major,75,64,52,41,0,11,3
9,Starboy,"The Weeknd, Daft Punk",2,2016,9,21,29536,79,2565529693,281,137,"2,445",1,140,186,G,Major,68,49,59,16,0,13,28


# Add new variables 

## Column track_id

In [6]:
# Create a function to obtain the ID of each song, based on its name
def get_track_id(track_name):
    try:
        results = sp.search(q=track_name, type='track', limit=1)
        track_id = results['tracks']['items'][0]['id']
        return track_id
    except:
        return None

# Apply the function get_track_id to every row in the column track_name
songs_apis_df['track_id'] = songs_apis_df['track_name'].apply(get_track_id)

In [7]:
# Show the entire DataFrame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
songs_apis_df

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,in_shazam_charts,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,track_id
0,Blinding Lights,The Weeknd,1,2019,11,29,43899,69,3703895074,672,199,"3,421",20,NaN,171,C#,Major,50,38,80,0,0,9,7,0VjIjW4GlUZAMYd2vXMi3b
1,Shape of You,Ed Sheeran,1,2017,1,6,32181,10,3562543890,33,0,"6,808",7,0,96,C#,Minor,83,93,65,58,0,9,8,7qiZfU4dY1lWllzX7mPBI3
2,Someone You Loved,Lewis Capaldi,1,2018,11,8,17836,53,2887241814,440,125,"1,800",0,NaN,110,C#,Major,50,45,41,75,0,11,3,7qEHsqek33rTcFNT9PFqLf
3,Dance Monkey,Tones and I,1,2019,5,10,24529,0,2864791672,533,167,"3,595",6,NaN,98,F#,Minor,82,54,59,69,0,18,10,2XU0oxnq2qxCpomAAuJY8K
4,Sunflower - Spider-Man: Into the Spider-Verse,"Post Malone, Swae Lee",2,2018,10,9,24094,78,2808096550,372,117,843,4,69,90,D,Major,76,91,50,54,0,7,5,0RiRZpuVRbi7oqRdSMwhQY
5,One Dance,"Drake, WizKid, Kyla",3,2016,4,4,43257,24,2713922350,433,107,"3,631",0,26,104,C#,Major,77,36,63,1,0,36,5,1zi7xx7UVEFkmKfv06H8x0
6,STAY (with Justin Bieber),"Justin Bieber, The Kid Laroi",2,2021,7,9,17050,36,2665343922,492,99,798,31,0,170,C#,Major,59,48,76,4,0,10,5,5HCyWlXZPP0y6Gqq8TgA20
7,Believer,Imagine Dragons,1,2017,1,31,18986,23,2594040133,250,121,"2,969",10,31,125,A#,Minor,77,74,78,4,0,23,11,0pqnGHJpmpxLKifKRmU6WP
8,Closer,"The Chainsmokers, Halsey",2,2016,5,31,28032,0,2591224264,315,159,"2,179",0,44,95,G#,Major,75,64,52,41,0,11,3,7BKLCZ1jbUBVqRi2FVlTVw
9,Starboy,"The Weeknd, Daft Punk",2,2016,9,21,29536,79,2565529693,281,137,"2,445",1,140,186,G,Major,68,49,59,16,0,13,28,7MXVkk9YMctZqd1Srtv4MB


## Split artist's names

In [26]:
# Separar los nombres de los artistas en columnas individuales
artists_df = songs_apis_df['artist(s)_name'].str.split(',', expand=True)

# Seleccionar solo las tres primeras columnas de artistas
artists_df = artists_df.iloc[:, :5]

# Renombrar las columnas con nombres más descriptivos
artists_df.columns = [f'artist_{i+1}_name' for i in range(artists_df.shape[1])]

# Eliminar las columnas de artistas originales del DataFrame original
songs_apis_df = songs_apis_df.drop(columns=['artist_1_name', 'artist_2_name', 'artist_3_name'], errors='ignore')

# Concatenar las nuevas columnas de artistas al DataFrame original
songs_apis_df = pd.concat([songs_apis_df, artists_df], axis=1)

In [27]:
# Drop artist_4_name, artist_5_name, artist_6_name, artist_7_name
songs_apis_df = songs_apis_df.drop(columns=['artist_6_name', 'artist_7_name'], errors='ignore')

In [28]:
# Show the entire DataFrame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
songs_apis_df

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,in_shazam_charts,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,track_id,duration,artist_1_name,artist_2_name,artist_3_name,artist_4_name,artist_5_name
0,Blinding Lights,The Weeknd,1,2019,11,29,43899,69,3703895074,672,199,"3,421",20,NaN,171,C#,Major,50,38,80,0,0,9,7,0VjIjW4GlUZAMYd2vXMi3b,200.040,The Weeknd,None,None,None,None
1,Shape of You,Ed Sheeran,1,2017,1,6,32181,10,3562543890,33,0,"6,808",7,0,96,C#,Minor,83,93,65,58,0,9,8,7qiZfU4dY1lWllzX7mPBI3,233.712,Ed Sheeran,None,None,None,None
2,Someone You Loved,Lewis Capaldi,1,2018,11,8,17836,53,2887241814,440,125,"1,800",0,NaN,110,C#,Major,50,45,41,75,0,11,3,7qEHsqek33rTcFNT9PFqLf,182.160,Lewis Capaldi,None,None,None,None
3,Dance Monkey,Tones and I,1,2019,5,10,24529,0,2864791672,533,167,"3,595",6,NaN,98,F#,Minor,82,54,59,69,0,18,10,2XU0oxnq2qxCpomAAuJY8K,209.438,Tones and I,None,None,None,None
4,Sunflower - Spider-Man: Into the Spider-Verse,"Post Malone, Swae Lee",2,2018,10,9,24094,78,2808096550,372,117,843,4,69,90,D,Major,76,91,50,54,0,7,5,0RiRZpuVRbi7oqRdSMwhQY,157.560,Post Malone,Swae Lee,None,None,None
5,One Dance,"Drake, WizKid, Kyla",3,2016,4,4,43257,24,2713922350,433,107,"3,631",0,26,104,C#,Major,77,36,63,1,0,36,5,1zi7xx7UVEFkmKfv06H8x0,173.986,Drake,WizKid,Kyla,None,None
6,STAY (with Justin Bieber),"Justin Bieber, The Kid Laroi",2,2021,7,9,17050,36,2665343922,492,99,798,31,0,170,C#,Major,59,48,76,4,0,10,5,5HCyWlXZPP0y6Gqq8TgA20,141.805,Justin Bieber,The Kid Laroi,None,None,None
7,Believer,Imagine Dragons,1,2017,1,31,18986,23,2594040133,250,121,"2,969",10,31,125,A#,Minor,77,74,78,4,0,23,11,0pqnGHJpmpxLKifKRmU6WP,204.346,Imagine Dragons,None,None,None,None
8,Closer,"The Chainsmokers, Halsey",2,2016,5,31,28032,0,2591224264,315,159,"2,179",0,44,95,G#,Major,75,64,52,41,0,11,3,7BKLCZ1jbUBVqRi2FVlTVw,244.960,The Chainsmokers,Halsey,None,None,None
9,Starboy,"The Weeknd, Daft Punk",2,2016,9,21,29536,79,2565529693,281,137,"2,445",1,140,186,G,Major,68,49,59,16,0,13,28,7MXVkk9YMctZqd1Srtv4MB,230.453,The Weeknd,Daft Punk,None,None,None


## Column artist_id

In [29]:
# Create a function to obtain the artist id based on his/her name
def obtener_id_artista(artist_name):
    try:
        results = sp.search(q='artist:' + artist_name, type='artist')
        artists = results['artists']['items']
        if artists:
            return artists[0]['id']
        else:
            return None
    except:
        return None

In [31]:
# Create a function to obtain the IDs of the artists for row
num_artist_columns = len(artists_df.columns)
def get_artist_ids(row):
    artist_ids = []
    for i in range(1, num_artist_columns + 1):
        artist_name = row[f'artist_{i}_name']
        artist_id = obtener_id_artista(artist_name)  
        if artist_id:
            artist_ids.append(artist_id)
    return artist_ids

# Aplicar la función a cada fila en la columna 'track_id' con tqdm
tqdm.pandas(desc="Calculando duración") 
# Apply the function to every row of the DataFrame
songs_apis_df['artist_ids'] = songs_apis_df.apply(get_artist_ids, axis=1)

In [38]:
# Convertir cada valor en la columna artist_ids en cadenas de texto
songs_apis_df['artist_ids'] = songs_apis_df['artist_ids'].astype(str)

# Eliminar [] de artist_ids
songs_apis_df['artist_ids'] = songs_apis_df['artist_ids'].str.replace('[', '').str.replace(']', '')

# Eliminar todas las comillas de artist_ids
songs_apis_df['artist_ids'] = songs_apis_df['artist_ids'].str.replace("'", '')

In [39]:
# Show the entire DataFrame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
songs_apis_df

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,in_shazam_charts,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,track_id,duration,artist_1_name,artist_2_name,artist_3_name,artist_4_name,artist_5_name,artist_ids
0,Blinding Lights,The Weeknd,1,2019,11,29,43899,69,3703895074,672,199,"3,421",20,NaN,171,C#,Major,50,38,80,0,0,9,7,0VjIjW4GlUZAMYd2vXMi3b,200.040,The Weeknd,None,None,None,None,1Xyo4u8uXC1ZmMpatF05PJ
1,Shape of You,Ed Sheeran,1,2017,1,6,32181,10,3562543890,33,0,"6,808",7,0,96,C#,Minor,83,93,65,58,0,9,8,7qiZfU4dY1lWllzX7mPBI3,233.712,Ed Sheeran,None,None,None,None,6eUKZXaKkcviH0Ku9w2n3V
2,Someone You Loved,Lewis Capaldi,1,2018,11,8,17836,53,2887241814,440,125,"1,800",0,NaN,110,C#,Major,50,45,41,75,0,11,3,7qEHsqek33rTcFNT9PFqLf,182.160,Lewis Capaldi,None,None,None,None,4GNC7GD6oZMSxPGyXy4MNB
3,Dance Monkey,Tones and I,1,2019,5,10,24529,0,2864791672,533,167,"3,595",6,NaN,98,F#,Minor,82,54,59,69,0,18,10,2XU0oxnq2qxCpomAAuJY8K,209.438,Tones and I,None,None,None,None,2NjfBq1NflQcKSeiDooVjY
4,Sunflower - Spider-Man: Into the Spider-Verse,"Post Malone, Swae Lee",2,2018,10,9,24094,78,2808096550,372,117,843,4,69,90,D,Major,76,91,50,54,0,7,5,0RiRZpuVRbi7oqRdSMwhQY,157.560,Post Malone,Swae Lee,None,None,None,"246dkjvS1zLTtiykXe5h60, 1zNqQNIdeOUZHb8zbZRFMX"
5,One Dance,"Drake, WizKid, Kyla",3,2016,4,4,43257,24,2713922350,433,107,"3,631",0,26,104,C#,Major,77,36,63,1,0,36,5,1zi7xx7UVEFkmKfv06H8x0,173.986,Drake,WizKid,Kyla,None,None,"3TVXtAsR1Inumwj472S9r4, 3tVQdUvClmAT7URs9V3rsp..."
6,STAY (with Justin Bieber),"Justin Bieber, The Kid Laroi",2,2021,7,9,17050,36,2665343922,492,99,798,31,0,170,C#,Major,59,48,76,4,0,10,5,5HCyWlXZPP0y6Gqq8TgA20,141.805,Justin Bieber,The Kid Laroi,None,None,None,"1uNFoZAHBGtllmzznpCI3s, 284HyOro5vrr8DXPrdPSQL"
7,Believer,Imagine Dragons,1,2017,1,31,18986,23,2594040133,250,121,"2,969",10,31,125,A#,Minor,77,74,78,4,0,23,11,0pqnGHJpmpxLKifKRmU6WP,204.346,Imagine Dragons,None,None,None,None,53XhwfbYqKCa1cC15pYq2q
8,Closer,"The Chainsmokers, Halsey",2,2016,5,31,28032,0,2591224264,315,159,"2,179",0,44,95,G#,Major,75,64,52,41,0,11,3,7BKLCZ1jbUBVqRi2FVlTVw,244.960,The Chainsmokers,Halsey,None,None,None,"69GGBxA162lTqCwzJG5jLp, 26VFTg2z8YR0cCuwLzESi2"
9,Starboy,"The Weeknd, Daft Punk",2,2016,9,21,29536,79,2565529693,281,137,"2,445",1,140,186,G,Major,68,49,59,16,0,13,28,7MXVkk9YMctZqd1Srtv4MB,230.453,The Weeknd,Daft Punk,None,None,None,"1Xyo4u8uXC1ZmMpatF05PJ, 7CTejuKlwb0KuJQP80OAOZ"


## Column duration

In [8]:
# Create a function to obtain the duration of each song based on its ID
def get_duration_with_progress(track_id):
    try:
        track_info = sp.track(track_id)
        duration_ms = track_info['duration_ms']
        # Convertir la duración a segundos
        duration_sec = duration_ms / 1000
        return duration_sec
    except:
        return None

# Aplicar la función a cada fila en la columna 'track_id' con tqdm
tqdm.pandas(desc="Calculando duración")  
songs_apis_df['duration'] = songs_apis_df['track_id'].progress_apply(get_duration_with_progress) 

Calculando duración:   0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
# Show the entire DataFrame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
songs_apis_df

In [45]:
# Save the cleaned DataFrame in a pickle file
songs_apis_df.to_pickle('songs_apis_df.pkl')